In [105]:
import time

import numpy as np
import glob
import os
from sklearn.neighbors import KNeighborsClassifier

In [106]:
# Directory containing all the phone swing data
folder_containing_sensor_data = './data/'
sensor_parse_interval_ms = 30

## Step 1: Data Loading

This section will load in the sensor data, as well as interpolate the data at a regular interval, which is defined above.

In [107]:
# Organize files that contain the relevant data for each swing type
data = {
    'acc': {
        'Forehand': {
            'Good': {},
            'Bad': {}
        },
        'Backhand': {
            'Good': {},
            'Bad': {}
        },
        'Overhead': {
            'Good': {},
            'Bad': {}
        }
    },
    'bar': {
        'Forehand': {
            'Good': {},
            'Bad': {}
        },
        'Backhand': {
            'Good': {},
            'Bad': {}
        },
        'Overhead': {
            'Good': {},
            'Bad': {}
        }
    }
}

In [108]:
def load_data_file(file_path):
    accel_mode = True
    if 'bar' in file_path:
        accel_mode = False

    loaded_file_data = np.genfromtxt(file_path, dtype='double', delimiter=',')
    loaded_file_data[:, 0] = loaded_file_data[:, 0] - loaded_file_data[0][0]

    new_timestamps = np.arange(0, loaded_file_data[:,0][-1], sensor_parse_interval_ms, dtype='int64')

    # Now, we interpolate the data to be on a common time axis
    if accel_mode:
        new_x = np.interp(x=new_timestamps, xp=loaded_file_data[:,0], fp=loaded_file_data[:,1])
        new_y = np.interp(x=new_timestamps, xp=loaded_file_data[:,0], fp=loaded_file_data[:,2])
        new_z = np.interp(x=new_timestamps, xp=loaded_file_data[:,0], fp=loaded_file_data[:,3])
        aligned_data = np.column_stack((new_timestamps, new_x, new_y, new_z))

    else:
        new_bar = np.interp(x=new_timestamps, xp=loaded_file_data[:,0], fp=loaded_file_data[:,1])
        aligned_data = np.column_stack((new_timestamps, new_bar))

    return aligned_data

In [109]:
all_data_files = glob.glob(os.path.join(folder_containing_sensor_data, '*'))

start_time = time.time()
for data_file in all_data_files:
    swing_success = 'Good'
    data_file_name_only = data_file.split('/')[-1][:-4]
    method, swing_type, timestamp = data_file_name_only.split('_')[:3]

    # Mark as bad if necessary
    if len(data_file_name_only.split('_')) >= 4:
        swing_success = 'Bad'

    data[method][swing_type][swing_success][timestamp] = load_data_file(data_file)
end_time = time.time()

print(f"Loaded Data from {len(all_data_files)//2} Swings in {(end_time-start_time):.2f} Seconds")

Loaded Data from 473 Swings in 1.09 Seconds


## Step 2: Classifier Creation

Now that the data has been recorded, it is time to create the classifier.